#Notebook KNN4: Bilderkennung





Wir wollen nun ein KNN konstruieren, welches handgeschriebene Ziffern erkennen soll. 
Sie werden sehen, wie einfach es ist, Ihr eigenes künstliches neuronales Netz zu definieren und trainieren zu lassen.


Hierbei verwenden wir den MNIST-Datensatz (Modified National Institute of Standards and Technology Database). Dieser Datensatz besteht aus handgeschriebenen Ziffern von Mitarbeitern des American Census Bureau (Volkszählungsbehörde in den USA) sowie von amerikanischen High School-Schülern (siehe https://en.wikipedia.org/wiki/MNIST_database).

Wir teilen die Daten in Trainingsdaten und Testdaten auf.
Das KNN soll nur mittels der Trainingsdaten lernen und anschließend überprüfen wir mit den Testdaten die Genauigkeit unseres Modells.



1.   Wir beschleunigen die Berechnungen in diesem Notebook, indem wir zusätzlich eine Grafikkarte verwenden: Gehen Sie auf "Runtime"-> "Change runtime type" und wählen Sie unter "Hardware accelerator" den Eintrag "GPU". Klicken Sie auf "Save". 
2.  Aktivieren Sie die folgende Zelle, um die nötigen Programmpakete und die Daten zu laden. Ignorieren Sie hierbei die im Code aufgeführten Befehle. Den Code aller weiteren Zellen werden Sie verstehen.





In [0]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
from matplotlib import pyplot as plt
from random import randint

  
import keras
from keras.datasets import mnist
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
  # siehe https://colab.research.google.com/drive/1U0sRZdxVUn8LbQN9KidMaomKt2MPJ2-o#forceEdit=true&offline=true&sandboxMode=true&scrollTo=XaR6ncOmHNNw
  #from keras.datasets import mnist
  #(x_train, y_train), (x_test, y_test) = mnist.load_data()
mnist = input_data.read_data_sets("MNIST_data/")
train_data = mnist.train.images  # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images  # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

# Trainings- und Testdaten


x_train = 100*train_data.reshape(len(train_data),28,28).astype('float32')
x_test = 100*eval_data.reshape(len(eval_data),28,28).astype('float32')

y_train = train_labels
y_test = eval_labels

y_train_hot = keras.utils.to_categorical(y_train, 10)
y_test_hot = keras.utils.to_categorical(y_test, 10)
  


Using TensorFlow backend.
W0626 14:49:30.225188 140478912391040 deprecation.py:323] From <ipython-input-1-db07df889231>:19: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0626 14:49:30.226850 140478912391040 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0626 14:49:30.228033 140478912391040 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a 

Extracting MNIST_data/train-images-idx3-ubyte.gz


W0626 14:49:30.519004 140478912391040 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0626 14:49:30.572117 140478912391040 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


#Der MNIST-Datensatz


Die folgenden Befehle zeigen uns an, wie viele Bilder in "x_train" und "x_test" gespeichert sind:



In [0]:
print("Anzahl der Trainingsdaten: " + str(len(x_train)))
print("Anzahl der Testdaten: " + str(len(x_test)))

Jedes einzelne Bild besteht aus 28 x 28 Pixeln. 

Wir lassen uns das 90ste Bild anzeigen, indem wir x_train[89] aufrufen. (Die Zählung beginnt bei 0, d.h. x_train[0] ist das erste Bild.)

Der Wert 0 steht hierbei für "weiß", der Wert 100 für "schwarz" und alle Werte dazwischen sind entsprechende Grautöne.

In [0]:
print("Ein Bild: \n" + str(x_train[89]))

Wir können uns das 28*28-Pixel-Bild mit folgendem Befehl zeigen lassen:

In [0]:
plt.imshow(x_train[89], cmap='Greys')
plt.show()

**Frage 1:**  Können Sie erraten, was der folgende Befehl macht, bevor Sie ihn ausführen?

In [0]:
plt.imshow(100-x_train[89], cmap='Greys')
plt.show()


Wir lassen uns 25 zufällige Bilder aus x_train anzeigen:

In [0]:
for i in range(25):
    ax = plt.subplot(5, 5, i+1)
    ax.axis('off')
    plt.imshow(x_train[randint(0, len(x_train))], cmap='Greys') 
plt.show()


In "y_train" und "y_test" befinden sich die zu erlerndenden Werte, nämlich die entsprechenden Ziffern. Sie sehen, dass es gar nicht so einfach ist, die korrekte Ziffer zu erkennen. 

In [0]:
print("Trainingsbild Nr. 1: ")
plt.imshow(x_train[8], cmap='Greys')
plt.show()
print("Ziffer von Bild Nr. 1: " + str(y_train[8]))

Wir schauen uns auch das erste Bild aus den Testdaten an:

In [0]:
print("Testbild Nr. 1: ")
plt.imshow(x_test[0], cmap='Greys')
plt.show()
print("Ziffer von Bild Nr. 1: " + str(y_test[0]))

**Frage 2:**  Können Sie erraten, was im folgenden Code berechnet wird, was also die Zahl z darstellt?

In [0]:
z = 0
for k in range(55000):
    if y_train[k] == 5:
        z = z + 1
print('z = ' + str(z))

#Modell Nr. 1


Die folgenden tensorflow-Befehle definieren ein neuronales Netz. Das KNN hat eine Input-Schicht, eine versteckte Schicht und eine Output-Schicht. Sie haben bereits das nötige Wissen, um alle Befehle nachvollziehen zu können!



**Frage 3:**  Aktivieren Sie die nächste Zelle. In der Zusammenfassung wird die Anzahl der zu erlernenden Parameter (=Gewichte) aufgelistet. Insgesamt gibt es 25153 Gewichte. Können Sie diese Zahl selbst berechnen? (Wie viele Neuronen hat jede Schicht? Denken Sie an zusätzliche Bias-Neuronen.)


In [0]:
# Modell definieren
model1 = Sequential()

model1.add(Dense(units=32, activation='sigmoid', use_bias=True, input_shape=(784,)))
model1.add(Dense(units=1, use_bias=True, activation='linear'))

model1.summary()

Wir trainieren nun unser KNN mittelse Gradientenverfahren und unserer üblichen Fehlerfunktion ('mean_squared_error').

Die Anzahl der Iterationen wird mit `epochs=500` festgelegt.

In jedem Schritt berechnen wir die Fehler nur für eine zufällig gewählte Menge von Daten.   Die Anzahl dieser Daten wird mit `batch_size=1000` festgelegt. (Das Optimierungsverfahren heißt aus diesem Grund "stochastisches Gradientenverfahren", was den Befehl  `optimizer="sgd" ` erklärt.)

In einer Graphik sehen wir die Prozentzahl der richtig zugeordneten Bilder, einmal für die Trainingsdaten, einmal für die Testdaten. Haben Sie für das Training etwas Geduld (5 Minuten bei `epochs = 500`?)

In [0]:
# Modell trainieren
Lernrate = 0.00001
batch_size = 1000
Anzahl_Iterationen = 10
model1.compile(optimizer=keras.optimizers.SGD(lr=Lernrate), loss='mean_squared_error', metrics=['accuracy'])
history = model1.fit(x_train.reshape(len(x_train),784), y_train, batch_size=batch_size, epochs=Anzahl_Iterationen, verbose=False, validation_data=(x_test.reshape(len(x_test),784), y_test))
loss, accuracy  = model1.evaluate(x_test.reshape(len(x_test),784), y_test, verbose=False)

# Ergbnis graphisch darstellen
plt.figure(figsize=(7,7))
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Modell-Genauigkeit')
plt.ylabel('Genauigkeit')
plt.xlabel('Iteration')
plt.legend(['Trainingsdaten', 'Testdaten'], loc='best')
plt.show()

print(f'Fehler bei den Testdaten: {loss:.3}')
print(f'Genauigkeit bei den Testdaten: {accuracy:.3}')


**Frage 4:**  Verändern Sie in der folgenden Zelle Modell 1 (Anzahl der Schichten, Neuronen, Lernrate, ...), um (in angemessener Zeit) ein besseres Ergebnis als das obige zu erzielen.

In [0]:
# Modell definieren
model1 = Sequential()

model1.add(Dense(units=32, activation='sigmoid', use_bias=True, input_shape=(784,)))
model1.add(Dense(units=1, use_bias=True, activation='linear'))

model1.summary()


# Modell trainieren
Lernrate = 0.00001
batch_size = 1000
Anzahl_Iterationen = 10
model1.compile(optimizer=keras.optimizers.SGD(lr=Lernrate), loss='mean_squared_error', metrics=['accuracy'])
# verbose=0: kein Output des Lernfortschritts, verbose=1: Output des Lernfortschritts
history = model1.fit(x_train.reshape(len(x_train),784), y_train, batch_size=batch_size, epochs=Anzahl_Iterationen, verbose=0, validation_data=(x_test.reshape(len(x_test),784), y_test))


# Ergbnis graphisch darstellen
plt.figure(figsize=(7,7))
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Modell-Genauigkeit')
plt.ylabel('Genauigkeit')
plt.xlabel('Iteration')
plt.legend(['Trainingsdaten', 'Testdaten'], loc='best')
plt.show()
loss, accuracy  = model1.evaluate(x_test.reshape(len(x_test),784), y_test, verbose=False)
print(f'Fehler bei den Testdaten: {loss:.3}')
print(f'Genauigkeit bei den Testdaten: {accuracy:.3}')

Wir testen Modell 1 an einem Beispielbild: 

In [0]:
Bild_nummer = 6990
print("Bild Nr. " + str(Bild_nummer) + ": ")
plt.imshow(x_train[Bild_nummer], cmap='Greys')
plt.show()
print("Ziffer von Bild Nr. " + str(Bild_nummer) + ": " + str(y_train[Bild_nummer]))
Vorhersage = model1.predict(x_train[Bild_nummer].reshape(1, 784))[0]
print("Vorhersage für Bild Nr. " + str(Bild_nummer) + ": " + str(Vorhersage[0]))

**Frage 5:** Wenn jedes Gewicht im KNN 32-Bit Speicherplatz benötigt, wie viele MB Speicherplatz benötigt dann in etwa Ihr KNN von Modell 1?


#Modell Nr. 2

Modell Nr. 1 besitzt folgendes Problem: Wird eine 4 fälschlicherweise als 5 vorhergesagt, beträgt der Fehler nur 1. Wird die 0 fälschlicherweise als 9 vorhergesagt, beträgt der Fehler 9. Also wird das KNN eher lernen, die kleinen von den großen Ziffern zu unterscheiden. Wir sollten also unsere Fehler-Funktion anpassen.

Hierzu verwenden wir eine andere Darstellung der Daten y_train und y_test, die in y_train_hot und y_test_hot gespeichert wurden. Eine Ziffer wird nun als 10-Vektor dargestellt. Hierbei entspricht die 0 dem Vektor [1,0,0,0,0,0,0,0,0,0], die 1 entspricht dem Vektor [0,1,0,0,0,0,0,0,0,0], usw. Diese Darstellung nennt man auch die 'One-Hot-Kodierung'.

An einem Beispiel sehen wir, wie die Ziffern nun dargestellt werden:


In [0]:
print("Bild Nr. 1: ")
plt.imshow(x_train[0], cmap='Greys')
plt.show()
print("Ziffer von Bild Nr. 1 in y_train : " + str(y_train[0]))

print("Ziffer von Bild Nr. 1 in y_train2: " + str(y_train_hot[0]))

Wir konstruieren nun ein KNN mit 10 Output-Neuronen o_0, ...., o_9. Erhalten wir z.B. den Output [0.1, 0, 0, 0, 0, 0.99, 0, 0, 0, 0], so interpretieren wir dies als Vorhersage für die Ziffer 5.

Wählen wir nun die Fehler-Funktion 'mean_squared_error', so berechnet sich der Fehler zwischen [a0, a1, a2, a3, a4, a5, a6, a7, a8, a9] und [b0, b1, b2, b3, b4, b5, b6, b7, b8, b9] als (a1-b1)^2 + ... + (a9-b9)^2. 

Der Fehler zwischen 4 und 5, also zwischen [0, 0, 0, 0, 1, 0, 0, 0, 0, 0] und [0, 0, 0, 0, 0, 1, 0, 0, 0, 0] ist also derselbe wie zwischen 0 und 9, also zwischen  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0] und [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]. (Der Fehler beträgt 2 in beiden Fällen)



**Frage 6:** Verändern Sie die übrigen Parameter (Schichten, Lernrate, etc.) in der nächsten Zelle, so dass sie Ihrem Modell 1 entsprechen und lassen Sie Modell 2 trainieren. Wie hat sich die Genauigkeit verändert?

In [0]:
# Modell definieren
model2 = Sequential()

model2.add(Dense(units=32, activation='sigmoid', use_bias=True, input_shape=(784,)))
model2.add(Dense(units=10, use_bias=True, activation='linear'))

model2.summary()

# Modell trainieren

Lernrate = 0.00001
batch_size = 1000
Anzahl_Iterationen = 10
model2.compile(optimizer=keras.optimizers.SGD(lr=Lernrate), loss='mean_squared_error', metrics=['accuracy'])
history = model2.fit(x_train.reshape(len(x_train),784), y_train_hot, batch_size=batch_size, epochs=Anzahl_Iterationen, verbose=False, validation_data=(x_test.reshape(len(x_test),784), y_test_hot))


# Ergbnis graphisch darstellen

plt.figure(figsize=(7,7))
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Modell-Genauigkeit')
plt.ylabel('Genauigkeit')
plt.xlabel('Iteration')
plt.legend(['Trainingsdaten', 'Testdaten'], loc='best')
plt.show()
loss, accuracy  = model2.evaluate(x_test.reshape(len(x_test),784), y_test_hot, verbose=False)
print(f'Fehler bei den Testdaten: {loss:.3}')
print(f'Genauigkeit bei den Testdaten: {accuracy:.3}')


#Modell Nr. 3

Wir machen erneut denselben Ansatz wie in Modell 2 und führen noch zwei Änderungen ein:



1.   Wir ändern die Output-Aktivierungsfunktion von 'linear' zu 'softmax'. Damit ist jeder Wert eines Output-Neurons zwischen 0 und 1 und die Summe der Werte der 10 Output-Neuronen ergibt stets 1. Die exakte Formel finden Sie hier: https://de.wikipedia.org/wiki/Softmax-Funktion Nun können wir eine Ausgabe wie z.B. [0.2, 0.1, 0, 0, 0, 0, 0, 0, 0, 0.7] als Wahrscheinlichkeiten interpretieren, d.h. das KNN sagt die Ziffer 9 mit Wahrscheinlichkeit 70% voraus, die Ziffer 0 mit Wahr. 20%, die Ziffer 1 mit Wahr. 10%.


2.   Wir verändern zudem noch die Fehlerfunktion und wählen 'categorical_crossentropy', d.h. die sogenannte Kreuzentropie. Diese Funktion eignet sich besser bei diskreten Werten mit softmax (Klassifikation) im Vergleich zu kontinuierlichen, siehe https://de.wikipedia.org/wiki/Kreuzentropie. Angenommen, ein Bild mit Ziffer 1, also [0,1,0,0,0,0,0,0,0,0] wird klassifiziert als [0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1]. Dann berechnet sich der Fehler bezüglich der Kreuzentropie als  -(0 * log(0.1) + 1 * log(0.1) + 0 * log(0.1) + ... + 0 * log(0.1)) = - log(0.1) = 2.30259...







In Modell 3 finden Sie nun die Aktivierungsfunktion 'softmax' und die Fehlerfunktion 'categorical_crossentropy'. 

**Frage 7:**  Können Sie die übrigen Parameter in Modell 3 so anpassen, dass Sie eine Genauigkeit von mind. 90% erreichen? 

In [0]:
# Modell definieren
model3 = Sequential()

model3.add(Dense(units=32, activation='sigmoid', use_bias=True, input_shape=(784,)))
model3.add(Dense(units=10, use_bias=True, activation='softmax'))

model3.summary()

# Modell trainieren
Lernrate = 0.001
batch_size = 1000
Anzahl_Iterationen = 10
model3.compile(optimizer=keras.optimizers.SGD(lr=Lernrate), loss='categorical_crossentropy', metrics=['accuracy'])
history = model3.fit(x_train.reshape(len(x_train),784), y_train_hot, batch_size=batch_size, epochs=Anzahl_Iterationen, verbose=False,  validation_data=(x_test.reshape(len(x_test),784), y_test_hot))


# Ergbnis graphisch darstellen
plt.figure(figsize=(7,7))
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Modell-Genauigkeit')
plt.ylabel('Genauigkeit')
plt.xlabel('Iteration')
plt.legend(['Trainingsdaten', 'Testdaten'], loc='best')
plt.show()
loss, accuracy  = model3.evaluate(x_test.reshape(len(x_test),784), y_test_hot, verbose=False)
print(f'Fehler bei den Testdaten: {loss:.3}')
print(f'Genauigkeit bei den Testdaten: {accuracy:.3}')


Sie können das Modell wieder an einzelnen Bildern testen:

In [0]:
Bild_nummer = 690
print("Bild Nr. " + str(Bild_nummer) + ": ")
plt.imshow(x_train[Bild_nummer], cmap='Greys')
plt.show()
print("Ziffer von Bild Nr. " + str(Bild_nummer) + ": " + str(y_train[Bild_nummer]))
Vorhersage = model3.predict(x_train[Bild_nummer].reshape(1, 784))[0]
print("Vorhersage für Bild Nr. " + str(Bild_nummer) + ": " + str(np.argmax(Vorhersage)) + " (mit Wahrscheinlichkeit " + "{0:0.2f}".format(100*Vorhersage[np.argmax(Vorhersage)])  + "%)")
print("\nWahrscheinlichkeiten: " + str(Vorhersage))

Sie können nun ein eigenes Bild hochladen. Es wird in das Format 28 x 28 und in Grauwerte umgewandelt. Danach wird das Bild dem trainierten Modell 3 übergeben und wir sehen, welche Ziffer das KNN im Bild sieht.

In [0]:
from google.colab import files
from keras.preprocessing import image
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
img_show = image.load_img(fn, target_size=(288, 288))
img = image.load_img(fn, target_size=(28, 28))
plt.figure(figsize=(5, 5))
plt.imshow(img_show, interpolation='nearest')
plt.axis('off')
plt.show()

from keras.preprocessing.image import save_img
# save the image with a new filename
save_img('test_picture.jpg', img)
import cv2
# Load in Grayscale
gray = cv2.imread('./test_picture.jpg', 0)
plt.figure(figsize=(5, 5))
plt.imshow(gray, cmap=plt.get_cmap('gray'), vmin=0, vmax=255)
plt.show()
Vorhersage = model3.predict(gray.reshape(1, 784))[0]
print("Vorhersage der Ziffer: "  + str(np.argmax(Vorhersage)))
print("Wahrscheinlichkeiten: " + str(Vorhersage))

#Modell Nr. 4 (für Fortgeschrittene)

Sie kennen bereits den Befehl

`model.add(Dense(units=1, activation='sigmoid', use_bias=True, input_shape=(2,)))`,

der einem neuronalen Netz einfach eine weitere Schicht hinzufügt. Dabei werden alle Neuronen mit allen Neuronen der Vorgänger-Schicht verknüpft. Das ist nicht immer sinnvoll und es gibt speziellere Schichten, teilweise mit anderen Funktionen als die bisher behandelten, teilweise mit weniger Verknüpfungen zur Vorgänger-Schicht.

Recherchieren Sie die Bedeutung der folgenden Befehle, konstruieren Sie Ihr eigenes Netz, trainieren Sie es und versuchen Sie eine möglichst gute Genauigkeit zu erzielen.

`model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))`

`model.add(MaxPooling2D(pool_size=(2, 2)))`

`model.add(Dropout(0.25))`

`model.add(Flatten())`








In [0]:
# Modell definieren
model4 = Sequential()

model4.add(Conv2D(32, kernel_size=(3, 3),  activation='relu', input_shape=(28, 28, 1)))
model4.add(MaxPooling2D(pool_size=(2, 2)))
model4.add(Dropout(0.25))
model4.add(Flatten())
model4.add(Dense(10, activation='softmax'))

model4.summary()


# Modell trainieren
Lernrate = 0.0001
batch_size = 500
Anzahl_Iterationen = 4
# Wir verwenden eine verbesserte Variante des Gradientenverfahren, nämlich "Adadelta".
# Alte Variante: model4.compile(optimizer=keras.optimizers.SGD(lr=Lernrate), loss='categorical_crossentropy', metrics=['accuracy'])
model4.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])
history = model4.fit(x_train.reshape(x_train.shape[0], 28, 28, 1), y_train_hot, batch_size=batch_size, epochs=Anzahl_Iterationen, verbose=False, validation_data=(x_test.reshape(x_test.shape[0], 28, 28, 1), y_test_hot))

# Ergbnis graphisch darstellen
plt.figure(figsize=(7,7))
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Modell-Genauigkeit')
plt.ylabel('Genauigkeit')
plt.xlabel('Iteration')
plt.legend(['Trainingsdaten', 'Testdaten'], loc='best')
plt.show()
loss, accuracy  = model4.evaluate(x_test.reshape(x_test.shape[0], 28, 28, 1), y_test_hot, verbose=False)
print(f'Fehler bei den Testdaten: {loss:.3}')
print(f'Genauigkeit bei den Testdaten: {accuracy:.3}')